![header](https://i.imgur.com/I4ake6d.jpg)
<div style="text-align: center;">
    <div style="max-width:50%; display:inline-block;">
        <img src="https://immerse-ocean.eu/img/headers/immerse-header-logo.png" style=""/> 
    </div>
    <div style="max-width:500px; display:inline-block;">
        <img src="https://www.unibo.it/it/immagini/1_UNIBO_Ateneo_vert_pos.jpg/@@images/51c830e4-97ca-4516-bdf8-64ea327bdab3.jpeg" style=""/> 
    </div>
</div>

# COPERNICUS MARINE Satellite SST Validation

***

<center><h1>How to use Satellite Sea Surface Temperature (SST) measurements to validate NEMO model simulations</h1></center>

***
# Table of contents
- [Introduction](#1\)-Introduction)
- [About the data](#2\)-About-the-data)
- [Jupyter notebook and required Python modules](#3\)-Jupyter-notebook-and-required-Python-modules)
- [Setup and Download Data](#4\)-Setup-and-Download-Data)
- [Read model and satellite SST Data](#5\)-Read-model-and-satellite-SST-Data)
- [Interpolate model SST on the satellite SST grid](#6\)-Interpolate-model-SST-on-the-satellite-SST-grid)
- [Exercise n.1: Generate model and satellite SST maps](#7\)-Exercise-n.1:-Generate-model-and-satellite-SST-maps)
- [Exercise n.2: Compute and plot RMSE between Model and Satellite SST](#8\)-Exercise-n.2:-Compute-and-plot-RMSE-between-Model-and-Satellite-SST)
***

# 1) Introduction

[Go back to the "Table of contents"](#Table-of-contents)

The objective of this exercise is to use satellite Sea Surface Temperature (SST) data  ([SST_MED_SST_L4_NRT_OBSERVATIONS_010_004](https://data.marine.copernicus.eu/product/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004) product) to perform the validation of the high-resolution nested model implemented in the Galicia coastal area (NW of Spain). The simulation started on 7 November 2002 at 00:00 and ran until 17 November 2002 at 24:00. The model is based on NEMO.v4.2 code and have been developed at [UNIBO](https://www.unibo.it/) in the context of the IMMERSE project.

In particular, we will:
- download and extract SST data from zenodo repository
- read model and satellite SST data
- interpolate the model SST on the satellite SST grid 
- generate model and satellite SST maps
- compute and plot the RMSE between measured and interpolated model data

# 2) About the data

[Go back to the "Table of contents"](#Table-of-contents)

## 2.1) Model SST Data

A NEMO configuration for the Galicia coastal area (NW of Spain) with resolution of ~2km was setup based on NEMO.v4.2 code. The model is initialized and forced at the open ocean boundaries via the CMEMS reanalysis fields 
([IBI_REANALYSIS_PHYS_005_002](https://data.marine.copernicus.eu/product/IBI_MULTIYEAR_PHY_005_002/description) product) with 1/12° horizontal resolution and receives its atmospheric forcing from ECMWF operational analyses, with a spatial resolution of 0.25° and 1-h temporal resolution. Model outputs are delivered with this exercise within the subfolder './data/child/' containing the land-sea mask and the SST netcdf files, corresponding to the period under consideration

### 2.2) Satellite SST Data 

The satellite SST datasets used for model validation are the daily gap-free (L4) [SST_MED_SST_L4_NRT_OBSERVATIONS_010_004](https://data.marine.copernicus.eu/product/SST_MED_SST_L4_NRT_OBSERVATIONS_010_004) product at (0.0417° x 0.0417°) horizontal resolution representative of night SST values (00:00 UTC). (Pisano et al 2016; doi:10.1016/j.rse.2016.01.019). The data are delivered with this exercise within the subfolder './data/sat/'

# 3) Jupyter notebook and required Python modules

[Go back to the "Table of contents"](#Table-of-contents)

To execute this notebook a jupyter notebook client is necessary which can be installed using 
[Anaconda Distribution](https://docs.anaconda.com/anaconda/install/index.html).
Anaconda Distribution includes Python, the Jupyter Notebook, and other commonly used packages for the scientific community.

**Requirement: Python version 3.7** onwards

Check your Python version with:

In [ ]:
!python --version

If needed you can install the right Python version in this way:
```
conda install python=3.7
```

The following Python modules are needed for running the exercises:

| Module name | Description |
| :---: | :---|
| **os** | [Miscellaneous operating system interfaces](https://docs.python.org/3.7/library/os.html) for managing paths, 
| **glob** | [Unix style pathname pattern expansion](https://docs.python.org/3.7/library/glob.html) this module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell |
| **math** | [math](https://docs.python.org/3/library/math.html) is a built-in module in the Python 3 standard library that provides standard mathematical constants and functions.  |
| **numpy** | [NumPy](https://numpy.org/) is the fundamental package for scientific computing with Python and for managing ND-arrays |
| **xarray** | [Xarray](http://xarray.pydata.org/en/stable/) introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like arrays, which allows for a more intuitive, more concise, and less error-prone developer experience. |
| **scipy** |[SciPy](https://docs.scipy.org/doc/scipy/index.html) is an open-source software for mathematics, science, and engineering. It includes modules for statistics, optimization, integration, linear algebra, and more. |
| **matplotlib** |[Matplotlib](https://matplotlib.org/) is a Python 2D plotting library which produces publication quality figures |
| **cartopy** |[Cartopy](https://scitools.org.uk/cartopy/docs/latest/) is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses. |
| **python-wget** |[python-wget] is a Python package designed to download remote files from the internet.

You may already have all the python packages you need.
If you are using conda as package manager, you can check by doing:
```
conda list
```

If some packages are missing you can install them into the current environment using the "conda install" command type the following:
```
conda install PACKAGES-LIST
```

# 4) Setup and Download Data

[Go back to the "Table of contents"](#Table-of-contents)

<div class="alert alert-block alert-warning">
    <h3>Before starting, read carefully the following execution notes</h3>
    <ul><li>
        Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play">
        </i></button> button from the icons menu above the notebook (otherwise press the keyboard shortcut `Shift` + `Enter`).
        </li>
    <li>
        If for any reason the notebook stops working, from the same menu click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the text menu above the icons one, click on "Cell" and select "Run All Above".
        </li></ul>
</div>

## Import the packages

In [ ]:
import os
from glob import glob
import math
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import FuncFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from scipy import interpolate

## Download and extract data

In [ ]:
# download data if not already there
print('download data')
try:
    import wget
except:
    print('wget not installed, please install it: conda install -c conda-forge python-wget') 
    
if (not os.path.exists('./data.zip')) and (not os.path.exists('./data')):
    url = 'https://zenodo.org/record/7516467/files/data.zip'
    filename = wget.download(url)
else:
    print('zip or data direcotry already present') 

In [ ]:
print('unzip data.zip')
try:
    import zipfile
except:            
    print('library zipfile missing data needs to be extracted manually')
        
if (os.path.exists('./data.zip')) and (not os.path.exists('./data')):  
    with zipfile.ZipFile('./data.zip', 'r') as zip_ref:
        zip_ref.extractall('./')        
        print('done extracting zip archiv')
else:
    print('zip or data direcotry already present')

## Inspect directories

In [ ]:
def checkDir(outPath):
    "function for creating paths, if needed"
    if not os.path.exists(outPath):
        os.makedirs(outPath)
        print('Creating output foler: '+str(outPath))
        
path_data = {}
path_data["root"] = "./data"
path_data["sat"]    = "./data/sat"
path_data["child"]  = "./data/child"

path_figure = {}
path_figure['root']  = "./figures"
path_figure['sat']   = "./figures/sat"
path_figure['child'] = "./figures/child"

#Check if directories exisit:
for diri in path_figure['root'],path_figure['sat'],path_figure['child']:
    checkDir(diri)

# 5) Read model and satellite SST Data

[Go back to the "Table of contents"](#Table-of-contents)

## Define file names

In [ ]:
ndays = 11

files = sorted([os.path.basename(filename) for filename in glob(path_data["child"]+"/"+"SURF_1h_*grid_T*")])
# print(files)
file_childmod_T = [fl for fl in files[0:ndays] ] #files[0:ndays]
date = [files[iday].split("_")[2] for iday in range(ndays)]
file_satsst = [str(date[iday])+"000000-GOS-L4_GHRSST-SSTfnd-OISST_HR_REP-MED-v02.0-fv02.0.nc" for iday in range(ndays)]
# file_parMod_T = ["oceBC_temp_"+dat+".nc" for dat in date]

file_meshmask = "mesh_mask.nc"

print(" --------------------------------------------------")
print(" | file_childmod_T = "+file_childmod_T[0])
if len(file_childmod_T)>1: [print(" |                   "+fl) for fl in file_childmod_T[1:]]
print(" | file_satsst = "+file_satsst[0])
if len(file_satsst)>1: [print(" |               "+fl) for fl in file_satsst[1:]]
print(" --------------------------------------------------")

## Read nested model SST data

In [ ]:
dimChild = {}
crdChild = {}
varChild = {}
dimChild["lon"] = "nav_lon"
dimChild["lat"] = "nav_lat"
dimChild["time"] = "time_counter"
crdChild["lon"] = "nav_lon"
crdChild["lat"] = "nav_lat"
crdChild["time"] = "time_counter"
varChild["sst"] = "votemper"

print("=== READ SPACE DIMENSION")
with xr.open_dataset(path_data["child"]+"/"+file_meshmask) as ds_childMesh:
    # print(ds_childMesh)

    nx_child = len(ds_childMesh[dimChild["lon"]][0,:])
    ny_child = len(ds_childMesh[dimChild["lat"]][:,0])

    print("   --------------------------------------------------")
    print("   |    nx_child = "+str(nx_child))
    print("   |    ny_child = "+str(ny_child))
    print("   --------------------------------------------------")
    
print("=== READ SPACE COORDIANTEs")
lon1dt_child = xr.DataArray(np.zeros((nx_child)),dims=[dimChild["lon"]])
lat1dt_child = xr.DataArray(np.zeros((ny_child)),dims=[dimChild["lat"]])
lon2dt_child = xr.DataArray(np.zeros((ny_child,nx_child)),dims=[dimChild["lat"],dimChild["lon"]])
lat2dt_child = xr.DataArray(np.zeros((ny_child,nx_child)),dims=[dimChild["lat"],dimChild["lon"]])
mask2dt_child = xr.DataArray(np.zeros((ny_child,nx_child)),dims=[dimChild["lat"],dimChild["lon"]])

with xr.open_dataset(path_data["child"]+"/"+file_meshmask) as ds_childMesh:
    # print(ds_childMesh)

    lon1dt_child[0:nx_child] = ds_childMesh["glamt"][0,0,0:nx_child]
    lat1dt_child[0:ny_child] = ds_childMesh["gphit"][0,0:ny_child,0]
    # print(lon1dt_child)

    # Builds 2D grids from 1D arrays
    xx,yy = np.meshgrid(lon1dt_child[0:nx_child],lat1dt_child[0:ny_child])
    lon2dt_child[0:ny_child,0:nx_child] = xr.DataArray(xx)
    lat2dt_child[0:ny_child,0:nx_child] = xr.DataArray(yy)

    mask2dt_child[0:ny_child,0:nx_child] = ds_childMesh["tmask"][0,0,0:ny_child,0:nx_child]
        
print("=== READ TIME-DIMENSION")

## Open multiple files as a single datasets with open_mfdataset() functions
with xr.open_mfdataset(list(map(lambda x: path_data["child"]+"/"+x, file_childmod_T))) as ds_child:
    # print(ds_child)
    nt_childM = ds_child.dims[dimChild["time"]]

    print("   --------------------------------------------------")
    print("   |    nt_childM = "+str(nt_childM))
    print("   --------------------------------------------------")

print("=== READ TIME-COORDIANTEs/VARs")
stimeYYYYMMDD_child = np.empty((nt_childM), dtype="object")
time_child = xr.DataArray(np.empty((nt_childM)),dims=[dimChild["time"]])
SST_child  = xr.DataArray(np.zeros((nt_childM,ny_child,nx_child)),dims=[dimChild["time"],dimChild["lat"],dimChild["lon"]])

## Open multiple files as a single datasets with open_mfdataset() functions
with xr.open_mfdataset(list(map(lambda x: path_data["child"]+"/"+x, file_childmod_T))) as ds_child:
    # print(ds_child)

    stimeYYYYMMDD_child[0:nt_childM] = ds_child[crdChild["time"]]
    
    SST_child[0:nt_childM,0:ny_child,0:nx_child] = np.where(mask2dt_child[0:ny_child,0:nx_child]==0, np.nan, ds_child[varChild["sst"]].isel(deptht=0))


## Read Satellite SST data

In [ ]:
dimObs = {}
crdObs = {}
varObs = {}
dimObs["lon"] = "lon"
dimObs["lat"] = "lat"
dimObs["time"] = "time"
crdObs["lon"] = "lon"
crdObs["lat"] = "lat"
crdObs["time"] = "time"
varObs["sst"] = "analysed_sst"

print("=== READ SPACE DIMENSION")
with xr.open_dataset(path_data["sat"]+"/"+file_satsst[0]) as ds_sat:
    # print(ds_sat)
    nx_obs = ds_sat.dims[dimObs["lon"]]
    ny_obs = ds_sat.dims[dimObs["lat"]]

    lon1dt_obs = ds_sat[crdObs["lon"]]
    lat1dt_obs = ds_sat[crdObs["lat"]]

    idx_lon0 = np.argwhere(np.array(lon1dt_obs[:].values)>lon1dt_child[0].values)[0][0]
    idx_lon1 = np.argwhere(np.array(lon1dt_obs[:].values)>lon1dt_child[nx_child-1].values)[0][0]-1

    idx_lat0 = np.argwhere(np.array(lat1dt_obs[:].values)>lat1dt_child[0].values)[0][0]

    idx_lat1 = np.argwhere(np.array(lat1dt_obs[:].values)>lat1dt_child[ny_child-1].values)[0][0]-1
    # print(idx_lat0)
    # print(idx_lat1)

    nx_subobs = (idx_lon1 - idx_lon0) + 1
    ny_subobs = (idx_lat1 - idx_lat0) + 1
    
    print(" --------------------------------------------------")
    print(" | nx_subobs = "+str(nx_subobs))
    print(" | ny_subobs = "+str(ny_subobs))
    print(" --------------------------------------------------")
    
print("=== READ TIME-DIMENSION")

# Open multiple files as a single datasets with open_mfdataset() functions
with xr.open_mfdataset(list(map(lambda x: path_data["sat"]+"/"+x, file_satsst))) as ds_sat:
    # print(ds_sat)
    nt_subobs = ds_sat.dims[dimObs["time"]]

print(" --------------------------------------------------")
print(" | nt_subobs ="+str(nt_subobs))
print(" --------------------------------------------------")

print("=== READ SPACE COORDIANTEs")
lon1dt_subobs  = xr.DataArray(np.zeros((nx_subobs)),dims=[dimObs["lon"]])
lat1dt_subobs  = xr.DataArray(np.zeros((ny_subobs)),dims=[dimObs["lat"]])
lon2dt_subobs  = xr.DataArray(np.zeros((ny_subobs,nx_subobs)),dims=[dimObs["lat"],dimObs["lon"]])
lat2dt_subobs  = xr.DataArray(np.zeros((ny_subobs,nx_subobs)),dims=[dimObs["lat"],dimObs["lon"]])
mask2dt_subobs = xr.DataArray(np.zeros((ny_subobs,nx_subobs)),dims=[dimObs["lat"],dimObs["lon"]])
SST_subobs     = xr.DataArray(np.zeros((nt_subobs,ny_subobs,nx_subobs)),dims=[dimObs["time"],dimObs["lat"],dimObs["lon"]])

time_subobs = xr.DataArray(np.empty((nt_subobs)),dims=[dimObs["time"]])
stimeYYYYMMDD_subobs = np.empty((nt_subobs), dtype="object")

 # Open multiple files as a single datasets with open_mfdataset() functions
with xr.open_mfdataset(list(map(lambda x: path_data["sat"]+"/"+x, file_satsst))) as ds_sat:
    # print(ds_sat)

    lon1dt_subobs[0:nx_subobs] = ds_sat[crdObs["lon"]].isel(lon=range(idx_lon0,idx_lon1+1))
    lat1dt_subobs[0:ny_subobs] = ds_sat[crdObs["lat"]].isel(lat=range(idx_lat0,idx_lat1+1))

    # Builds 2D grids from 1D arrays
    xx,yy = np.meshgrid(lon1dt_subobs[0:nx_subobs],lat1dt_subobs[0:ny_subobs])
    lon2dt_subobs[0:ny_subobs,0:nx_subobs] = xr.DataArray(xx)
    lat2dt_subobs[0:ny_subobs,0:nx_subobs] = xr.DataArray(yy)

    time_subobs[0:nt_subobs] = ds_sat[crdObs["time"]]
    stimeYYYYMMDD_subobs[0:nt_subobs] = np.datetime_as_string(ds_sat[crdObs["time"]], unit='m')

    SST_subobs[0:nt_subobs,0:ny_subobs,0:nx_subobs] = ds_sat[varObs["sst"]].isel(lat=range(idx_lat0,idx_lat1+1)).isel(lon=range(idx_lon0,idx_lon1+1)) - 273.15

    mask2dt_subobs[0:ny_subobs,0:nx_subobs] = xr.where(np.isnan(SST_subobs[0,0:ny_subobs,0:nx_subobs]), 0., 1.)


# 6) Interpolate model SST on the satellite SST grid 

[Go back to the "Table of contents"](#Table-of-contents)

## Compute (time-averaged) nested model SST

The satellite SST data are representative of night SST values (centered at 00:00 UTC). So, before comparing measured and model data, we average over time (from 21:00 pm to 6:00 am) the model SST.

In [ ]:
SST_childAv = xr.DataArray(np.zeros((ndays,ny_child,nx_child)), dims=[dimObs["time"],dimChild["lat"],dimChild["lon"]])

for iday in range(ndays):

    print(" Day("+str(iday)+"):"+stimeYYYYMMDD_subobs[iday])

    # le medie SSTMOD-CHILD dalle 21:00 alle 6:00
    if(iday == 0):
        itimeSST = 0
        dtimeSST = 6
    elif(iday == 1):
        itimeSST = 21
        dtimeSST = 9
    else:
        itimeSST = itimeSST + 24
        dtimeSST = 9
    # print(list(range(itimeSST,itimeSST+dtimeSST)))
    somma2D = xr.DataArray(np.zeros((ny_child,nx_child)),dims=[dimChild["lat"],dimChild["lon"]])
    for itime in range(itimeSST,itimeSST+dtimeSST):
        somma2D[0:ny_child,0:nx_child] = somma2D[0:ny_child,0:nx_child] +  SST_child[itime,0:ny_child,0:nx_child]

    SST_childAv[iday,0:ny_child,0:nx_child] = somma2D[0:ny_child,0:nx_child] / dtimeSST


## Interpolate nested model SST on the Satellite SST Grid

In [ ]:
SST_childAv_grObs = xr.DataArray(np.zeros((ndays,ny_subobs,nx_subobs)),dims=[dimObs["time"],dimObs["lat"],dimObs["lon"]])

lon2dt_child_np = lon2dt_child.values
lat2dt_child_np = lat2dt_child.values
for iday in range(ndays):

    print(" Day("+str(iday)+"):"+stimeYYYYMMDD_subobs[iday])
    
    SST_childAv_np = SST_childAv.values[iday,:]
    # Interpolate SST from ChildGrid to Satellite grid points
    SST_childAv_grObs[iday,:]=interpolate.griddata((lon2dt_child_np.flatten(),lat2dt_child_np.flatten()),
                     SST_childAv_np.flatten(), (lon2dt_subobs,lat2dt_subobs), method='linear')

# 7) Exercise n.1: Generate model and satellite SST maps

[Go back to the "Table of contents"](#Table-of-contents)

 ### Define general plotting functions

In [ ]:
def plot_2Dfield_xy(filename,topString,colorBar,
                    nlon,nlat,lon2d,lat2d,fieldxy,
                    proj,lbOrient,labTitleStr,
                    minLatF,maxLatF,minLonF,maxLonF,
                    minLevelVal,maxLevelVal,levelSpacing):

    # Define the figure size
    mydpi = 100 # (dots per inch) where 1 dot is one pixel
    w_in, h_in = 10, 10
    fig_size = (w_in, h_in) #size in inches

    if proj == 'platecarree':
        projection = ccrs.PlateCarree()
        
    # Define the figure and each axis for the n-rows and m-columns
    fig, axs = plt.subplots(subplot_kw={'projection': projection},
                              figsize=fig_size)
    
    res = '10m'

    # Adds coastlines to the current axes
    axs.coastlines(resolution=res, linewidths=0.5)# Turn on continent shading
    axs.add_feature(cfeature.LAND.with_scale(res),facecolor='lightgray')
    axs.add_feature(cfeature.COASTLINE)

    # Set a title for the axes.
    axs.set_title(topString, fontsize=13)

    # Add grid lines to the current axes
    gl = axs.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                       linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlines = False
    gl.ylines = False
    gl.xlabel_style = {'size': 11, 'color': 'black'}
    gl.ylabel_style = {'size': 11, 'color': 'black'}

    # Add ticks-location, ticks-labels and their styles
    dx_ticks=0.5
    dy_ticks=0.5
    xticks=np.arange(math.floor(minLonF), math.ceil(maxLonF), dx_ticks)
    yticks=np.arange(math.floor(minLatF), math.ceil(maxLatF), dy_ticks)
    axs.set_xticks(xticks, crs=ccrs.PlateCarree())
    axs.set_yticks(yticks, crs=ccrs.PlateCarree())
    axs.set_xticklabels(xticks, rotation=0, fontsize=11)
    axs.set_yticklabels(yticks, rotation=0, fontsize=11)
    lon_formatter = cticker.LongitudeFormatter()
    lat_formatter = cticker.LatitudeFormatter()
    axs.xaxis.set_major_formatter(lon_formatter)
    axs.yaxis.set_major_formatter(lat_formatter)

    # plot contours
    nlevels = math.floor((maxLevelVal-minLevelVal)/levelSpacing)

    levels = np.linspace(minLevelVal, maxLevelVal, nlevels)
    scalar = axs.contourf(lon2d[0:nlat,0:nlon], lat2d[0:nlat,0:nlon], fieldxy[0:nlat,0:nlon],
                          levels=levels, cmap=colorBar,extend='both')
    
    # add colorbar
    axins = inset_axes(axs,width="100%", height="5%",loc='lower center',borderpad=-5)
    fmt = lambda x, pos: '{:,.2f}'.format(x)
    cbar = fig.colorbar(scalar, cax=axins, orientation=lbOrient, format=FuncFormatter(fmt), ax=fig.axes)
    cbar.ax.set_title(labTitleStr)

    # Set the map extension (x0,x1,y0,y1) in the given coord. system
    axs.set_extent([minLonF,maxLonF,minLatF,maxLatF])
    
    # Display the figure
    # plt.show()
    
    # Save the current figure to a file
    fig.savefig(filename+'.png',bbox_inches='tight',dpi=mydpi)

    # Close a window,
    plt.close(fig)

### Configure the plot

In [ ]:
minLonF = np.min(lon1dt_child)
maxLonF = np.max(lon1dt_child)
minLatF = np.min(lat1dt_child)
maxLatF = np.max(lat1dt_child)

colorBarOce = {}
colorBarOce['temp_colorTab']='jet' #rainbow
colorBarOce['temp_rangexy_min']  = 14.0
colorBarOce['temp_rangexy_max']  = 19.0
colorBarOce['temp_rangexy_delta']= 0.2

lbTitle = "Temperature [C]"
mapProj = 'platecarree'
lbOrient   = 'horizontal'


### Generate Satellite-SST maps

In [ ]:
base_name = "sstObs"
for iday in range(ndays):
    filename   = base_name + "_t"+'{:03d}'.format(iday)
    filepath_img = path_figure['sat']+"/"+filename
    if os.path.isfile(filepath_img+".png"): os.remove(filepath_img+".png")
    topString =  "Day("+str(iday)+")="+str(stimeYYYYMMDD_subobs[iday]) + "  range=["+ \
                  "{:.2f}".format(np.nanmin(SST_subobs[iday,0:ny_subobs,0:nx_subobs].values))+", "+\
                  "{:.2f}".format(np.nanmax(SST_subobs[iday,0:ny_subobs,0:nx_subobs].values))+"]"
    print(" ### SST Satellite  Day("+str(iday)+"):"+str(stimeYYYYMMDD_subobs[iday]))

    plot_2Dfield_xy(filepath_img,topString,colorBarOce['temp_colorTab'],
                    nx_subobs,ny_subobs,
                    lon2dt_subobs.isel(lat=range(ny_subobs)).isel(lon=range(nx_subobs)),
                    lat2dt_subobs.isel(lat=range(ny_subobs)).isel(lon=range(nx_subobs)),
                    SST_subobs.isel(time=iday).isel(lat=range(ny_subobs)).isel(lon=range(nx_subobs)),
                    mapProj,lbOrient,lbTitle,
                    minLatF,maxLatF,minLonF,maxLonF,
                    colorBarOce['temp_rangexy_min'],colorBarOce['temp_rangexy_max'],colorBarOce['temp_rangexy_delta'])


### Generate (time-averaged) nested model SST

In [ ]:
base_name = "sstChildAv"
for iday in range(ndays):
    filename   = base_name + "_t"+'{:03d}'.format(iday)
    filepath_img = path_figure["child"]+"/"+filename
    if os.path.isfile(filepath_img+".png"): os.remove(filepath_img+".png")
    topString =  "Day("+str(iday)+")="+str(stimeYYYYMMDD_child[iday]) + "  range=["+ \
                  "{:.2f}".format(np.nanmin(SST_child[0:ny_child,0:nx_child].values))+", "+\
                  "{:.2f}".format(np.nanmax(SST_child[0:ny_child,0:nx_child].values))+"]"
    
    print(" ### SST Child-Model Average[21:00-6:00]  Day("+str(iday)+"):"+str(stimeYYYYMMDD_child[iday]))
    colorBarOce['temp_colorTab']='jet' #rainbow
    plot_2Dfield_xy(filepath_img,topString,colorBarOce['temp_colorTab'],
                    nx_child,ny_child,
                    lon2dt_child,
                    lat2dt_child,
                    SST_childAv.isel(time=iday),
                    mapProj,lbOrient,lbTitle,
                    minLatF,maxLatF,minLonF,maxLonF,
                    colorBarOce['temp_rangexy_min'],colorBarOce['temp_rangexy_max'],colorBarOce['temp_rangexy_delta'])


### Generate (time-averaged) nested model SST on Satellite-Grid

In [ ]:
base_name = "sstChildAV_grObs"
for iday in range(ndays):
    filename   = base_name + "_t"+'{:03d}'.format(iday)
    filepath_img = path_figure["child"]+"/"+filename
    if os.path.isfile(filepath_img+".png"): os.remove(filepath_img+".png")
    topString =  "Day("+str(iday)+")="+str(stimeYYYYMMDD_subobs[iday]) + "  range=["+ \
                  "{:.2f}".format(np.nanmin(SST_childAv_grObs[iday,0:ny_child,0:nx_child].values))+", "+\
                  "{:.2f}".format(np.nanmax(SST_childAv_grObs[iday,0:ny_child,0:nx_child].values))+"]"
    print(" ### SST Child-Model Average[21:00-6:00] on Satellite-Grid  Day("+str(iday)+"):"+str(stimeYYYYMMDD_subobs[iday]))      
    
    plot_2Dfield_xy(filepath_img,topString,colorBarOce['temp_colorTab'],
                    nx_subobs,ny_subobs,
                    lon2dt_subobs.isel(lat=range(ny_subobs)).isel(lon=range(nx_subobs)),
                    lat2dt_subobs.isel(lat=range(ny_subobs)).isel(lon=range(nx_subobs)),
                    SST_childAv_grObs.isel(time=iday).isel(lat=range(ny_subobs)).isel(lon=range(nx_subobs)),
                    mapProj,lbOrient,lbTitle,
                    minLatF,maxLatF,minLonF,maxLonF,
                    colorBarOce['temp_rangexy_min'],colorBarOce['temp_rangexy_max'],colorBarOce['temp_rangexy_delta'])


# 8) Exercise n.2: Compute and plot RMSE between Model and Satellite SST

[Go back to the "Table of contents"](#Table-of-contents)

## Compute Root-Mean-Square-Error (Model VS Satellite)

In [ ]:
rmseSST_childAv = np.full([ndays], np.nan)

for iday in range(ndays):

    print(" ### Day("+str(iday)+"):"+stimeYYYYMMDD_subobs[iday])
    
    rmseSST_childAv_temp = 0.
    N=0
    for iy in range(ny_subobs):
        for ix in range(nx_subobs):
            if (( not np.isnan(SST_childAv_grObs[iday,iy,ix])) and ( not np.isnan(SST_subobs[iday,iy,ix]))):
                N=N+1
                rmseSST_childAv_temp = rmseSST_childAv_temp + (SST_childAv_grObs[iday,iy,ix]-SST_subobs[iday,iy,ix])**2
                # print(f"iy={iy} ix={ix} N={N} SST_childAv_grObs={SST_childAv_grObs[iday,iy,ix]} SST_subobs={SST_subobs[iday,iy,ix]} rmseSST_childAv_temp={rmseSST_childAv_temp}")
    if (N > 0): rmseSST_childAv[iday] = np.sqrt(rmseSST_childAv_temp/N)

    print("     rmseSST_childAv="+str(rmseSST_childAv[iday]) )
        


 ### Define general plotting functions

In [ ]:
def plot_1Dfield(filename,topString,
                nx,xArray,yArray,
                xminF,xmaxF,yminF,ymaxF,
                xAxisLab,yAxisLab,
                lineColor,legendLab,legendPos,yReverse):

    # Define the figure size
    mydpi = 100 # (dots per inch) where 1 dot is one pixel
    w_in, h_in = 10, 10
    fig_size = (w_in, h_in) #size in inches

    fig, axs = plt.subplots(figsize=fig_size)

    # Plot y versus x as lines for each Axes
    axs.plot(xArray, yArray, linestyle='-', marker='o', color=lineColor, label=legendLab)
    # axs.plot(xArray, yArray)

    # Reverse the y axis
    if (yReverse): axs.invert_yaxis()

    #set tick labels font to size 20
    axs.tick_params(axis='x', labelsize=15)
    axs.tick_params(axis='y', labelsize=15)

    # Add a legend on the Axes
    axs.legend(loc=legendPos, fontsize=15)

    # Set title and x-y axis labels for the Axes
    axs.set_title(topString, fontsize=15)
    axs.set_xlabel(xAxisLab, fontsize=15)
    axs.set_ylabel(yAxisLab, fontsize=15)

    # Set the y-axis view limits for the Axes
    axs.set_ylim(yminF, ymaxF)

    # Display the figure
    # plt.show()
    
    # Save the current figure to a file
    fig.savefig(filename+'.png',bbox_inches='tight',dpi=mydpi)

    # Close a window,
    plt.close(fig)

### Configure the plot

In [ ]:
xAxisLab  = "Date"
yAxisLab  = "RMSE"
legendLab = "RMSE ChildAv-Sat"
yReverse  = False
legendPos =  "upper right" #"upper left", "lower left", "center right"
colorLine = "red"
xminF = 1
xmaxF = 2
yminF = 0.4
ymaxF = 0.85

 ### Plot RMSE (Model VS Satellite)

In [ ]:
filename   = "RMSE_childAv-Sat"
filepath_img = path_figure["child"]+"/"+filename
if os.path.isfile(filepath_img+".png"): os.remove(filepath_img+".png")
topString = "Root Mean Square Error - NEST1" + "  range=["+ \
                  "{:.2f}".format(np.nanmin(rmseSST_childAv[0:ndays]))+", "+\
                  "{:.2f}".format(np.nanmax(rmseSST_childAv[0:ndays]))+"]"

yArray = rmseSST_childAv[0:ndays]
xArray = [np.array(stimeYYYYMMDD_subobs[it], dtype=np.datetime64) for it in range(nt_subobs)]

plot_1Dfield(filepath_img,topString,
             ndays,xArray,yArray,
             xminF,xmaxF,yminF,ymaxF,xAxisLab,yAxisLab,
             colorLine,legendLab,legendPos,yReverse)